In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from google.colab.patches import cv2_imshow


In [ ]:
from google.colab import files
uploaded = files.upload()

# Membaca gambar
filename = next(iter(uploaded))
img = cv2.imread(filename)

# Konversi ke grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Tampilkan citra asli
plt.figure(figsize=(5, 5))
plt.imshow(gray, cmap='gray')
plt.title("Citra Asli (Gelap)")
plt.axis('off')
plt.show()


***CONTRAST STRETCHING***

In [ ]:
def contrast_stretching(image):
    r_min = np.min(image)
    r_max = np.max(image)
    stretched = (image - r_min) * (255 / (r_max - r_min))
    stretched = np.clip(stretched, 0, 255).astype(np.uint8)
    return stretched

stretched_img = contrast_stretching(gray)

plt.figure(figsize=(5, 5))
plt.imshow(stretched_img, cmap='gray')
plt.title("Hasil Contrast Stretching")
plt.axis('off')
plt.show()


***HISTROGRAM EQUALIZATION***

In [ ]:
equalized_img = cv2.equalizeHist(gray)

plt.figure(figsize=(5, 5))
plt.imshow(equalized_img, cmap='gray')
plt.title("Hasil Histogram Equalization")
plt.axis('off')
plt.show()



***FUNGSI PSNR & STANDAR DEVIASI***

In [ ]:
def calculate_psnr(original, processed):
    mse = np.mean((original - processed) ** 2)
    if mse == 0:
        return float('inf')  # tidak ada perbedaan
    max_pixel = 255.0
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

# Hitung PSNR
psnr_stretch = calculate_psnr(gray, stretched_img)
psnr_equal = calculate_psnr(gray, equalized_img)

# Hitung standar deviasi
std_stretch = np.std(stretched_img)
std_equal = np.std(equalized_img)

print(f"PSNR Contrast Stretching: {psnr_stretch:.2f} dB")
print(f"PSNR Histogram Equalization: {psnr_equal:.2f} dB\n")

print(f"Standar Deviasi Contrast Stretching: {std_stretch:.2f}")
print(f"Standar Deviasi Histogram Equalization: {std_equal:.2f}")


***PERBANDINGAN HISTOGRAM***

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.hist(gray.ravel(), bins=256, color='gray')
plt.title('Histogram Citra Asli')

plt.subplot(1, 3, 2)
plt.hist(stretched_img.ravel(), bins=256, color='blue')
plt.title('Histogram Contrast Stretching')

plt.subplot(1, 3, 3)
plt.hist(equalized_img.ravel(), bins=256, color='green')
plt.title('Histogram Histogram Equalization')

plt.tight_layout()
plt.show()


In [ ]:

results = {
    "Metode": ["Original", "Contrast Stretching", "Histogram Equalization"],
    "Standar Deviasi": [np.std(gray), std_stretch, std_equal],
    "PSNR (dB)": ["-", f"{psnr_stretch:.2f}", f"{psnr_equal:.2f}"]
}

df = pd.DataFrame(results)
print("\nTabel Perbandingan Metode:")
print(df.to_markdown(index=False))

In [ ]:
def detect_noise(image):
    # Deteksi piksel ekstrem (0 atau 255) sebagai indikator noise
    noise_pixels = np.sum((image == 0) | (image == 255))
    noise_percentage = (noise_pixels / image.size) * 100
    return noise_percentage

noise_equal = detect_noise(equalized_img)
print(f"\nPersentase Noise (Histogram Equalization): {noise_equal:.2f}%")

In [ ]:
print("\nKESIMPULAN:")
if std_equal > std_stretch:
    print("- Histogram Equalization lebih unggul dalam meningkatkan kontras (standar deviasi lebih tinggi).")
else:
    print("- Contrast Stretching lebih unggul dalam meningkatkan kontras.")

if psnr_stretch > psnr_equal:
    print("- Contrast Stretching lebih baik dalam menjaga kualitas citra (PSNR lebih tinggi).")
else:
    print("- Histogram Equalization lebih baik dalam menjaga kualitas citra.")